In [4]:
############################
##### LOAD LIBRARIES #######
############################

import os
import rasterio
from rasterio.windows import Window
import pandas as pd
import tarfile
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import xml.etree.ElementTree as ET

In [19]:
############################
###### LOCATION SPECS ######
############################

# path and row should change for each iteration
path = 170
row_name = '064'
path_row = "170064"
#wd = "C:/Users/asobc/PycharmProjects/fallow/"
wd = "D:/"
#folder_name = f'{path}_{row_name}_raw/'
folder_name = f'{path}_{row_name}_raw/'
folder_path = os.path.join(wd, folder_name)
plant_time = ["02","03"]
grow_time = ["04","05","06"]
growing_mid_month = '06'
growing_mid_day = '15'
planting_mid_month = '02'
planting_mid_day = '15'

In [5]:
############################
##### DEFINE FUNCTIONS #####
############################

# extracting date from file name
def extract_date(filename):
    return filename[17:25]

# converting utm to lon and lat
def utm_to_latlon(row, east_name, north_name, zone_number):
    easting = row[east_name]
    northing = row[north_name]

    utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', datum='WGS84')

    lon, lat = utm_proj(easting, northing, inverse=True)

    return pd.Series({'lat': lat, 'lon': lon})
# converting lon and lat to utm
def latlon_to_utm(longitude, latitude, zone_number):
    utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', datum='WGS84')

    easting, northing = utm_proj(longitude, latitude)

    return easting, northing

# getting the utm zone code out of the .xml file
def extract_utm_zone(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    zone_tag = root.find('.//{http://espa.cr.usgs.gov/v2}zone_code')

    if zone_tag is not None:
        zone_code = zone_tag.text.strip()
        return zone_code
    else:
        return None

# getting the utm zone code out of the .xml file
def extract_scene_coords(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    corner_coordinates = {}

    for corner_point in root.findall('.//{http://espa.cr.usgs.gov/v2}corner_point'):
        location = corner_point.get('location')
        x = float(corner_point.get('x'))
        y = float(corner_point.get('y'))
        corner_coordinates[location + '_x'] = x
        corner_coordinates[location + '_y'] = y

    return corner_coordinates

# trimming the raster
# REVISIT
# Commented areas are for plotting the raster
def trim_raster(raster_file, trim_box, output_folder, note, season):
    with rasterio.open(raster_file) as src:
        window = src.window(*trim_box)
        data = src.read(window=window)
        #bounds = src.window_bounds(window)
        #lon, lat = np.meshgrid(np.linspace(bounds[0], bounds[2], data.shape[2]),
        #                       np.linspace(bounds[1], bounds[3], data.shape[1]))

        transform = src.window_transform(window)

        meta = src.meta.copy()
        meta.update({
            'transform': transform,
            'height': window.height,
            'width': window.width
        })
        output_file = os.path.join(output_folder,os.path.basename(raster_file)[:-4] + f'_{season}_{note}_TRIM.tif')

        # Write the trimmed raster to the output file
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(data)

    pass

    # plt.imshow(data.squeeze(), cmap='jet', extent = [lon.min(), lon.max(), lat.min(), lat.max()])
    # plt.title('Zoomed in Bounding Box')
    # plt.xlabel('lon (meters)')
    # plt.ylabel('lat (meters)')
    # plt.colorbar(label='EVI')
    # plt.show()

# converting trimmed rasters to csvs
def raster_to_csv(raster_file, quality_file, east_name, north_name, zone_number, output_folder):

    with rasterio.open(raster_file) as src:
        raster_array = src.read(1)
        nodata_value = src.nodata
        raster_array = np.where(raster_array == nodata_value, np.nan, raster_array)
        transform = src.transform

        rows, cols = raster_array.shape
        X, Y = np.meshgrid(np.arange(0, cols), np.arange(0, rows))
        east, north = transform * (X, Y)

        flat_east = east.flatten()
        flat_north = north.flatten()
        flat_data = raster_array.flatten()

        evi = pd.DataFrame({
            'easting': flat_east,
            'northing': flat_north,
            'evi': flat_data
        })
    with rasterio.open(quality_file) as src:
        quality_array = src.read(1)

        nodata_value = src.nodata
        quality_array = np.where(quality_array == nodata_value, np.nan, quality_array)
        transform = src.transform
        rows, cols = quality_array.shape
        X_q, Y_q = np.meshgrid(np.arange(0, cols), np.arange(0, rows))

        east_q, north_q = transform * (X_q, Y_q)

        flat_east_q = east_q.flatten()
        flat_north_q = north_q.flatten()
        flat_data_q = quality_array.flatten()

        qual = pd.DataFrame({
            'easting': flat_east_q,
            'northing': flat_north_q,
            'quality': flat_data_q
        })

    df = pd.merge(evi, qual, on=['easting', 'northing'])

    del evi
    del qual
    df[['latitude', 'longitude']] = df.apply(utm_to_latlon, axis=1, args=(east_name, north_name, zone_number))

    output_file = os.path.join(output_folder,os.path.basename(raster_file)[:-11] + '.csv')
    df.to_csv(output_file, index=False)

    pass

In [70]:
###########################
###### EXTRACT FILES ######
###########################

def extract_specific_files(file_path, destination, evi, quality, xml):
    with tarfile.open(file_path, 'r:gz') as tar:
        # tar.extractall(destination)
        for member in tar.getmembers():
            if member.name.endswith(evi) or member.name.endswith(quality) or member.name.endswith(xml):
                member.name = os.path.basename(member.name)
                path = os.path.join(destination, member.name)
                if os.path.exists(path):
                    continue
                else:
                    tar.extract(member, destination)

            else:
                continue
    pass
for file in os.listdir(folder_path):
    name, ext = os.path.splitext(file)
    if ext == ".gz":
        file_name = name
        ext = ext
        extract_specific_files(f'{folder_path}{file_name}{ext}', folder_path, "_SR_EVI.tif","_QA_PIXEL.tif", "T1.xml")
    else:
        continue

In [8]:
##########################
##### FILE DATAFRAME #####
##########################

file_data = {}
for filename in os.listdir(folder_path):
    name, ext = os.path.splitext(filename)
    date = extract_date(name)
    # Check if the file is a TIFF file
    if ext == ".tif" and 'EVI' in name:
        file_type = "evi"
    elif ext == ".tif" and 'PIXEL' in name:
        file_type = "quality"
    elif ext == ".xml" and 'MTL' not in name:
        file_type = "xml"
    else:
        continue  # Skip files of other types
    if date in file_data:
        file_data[date][file_type] = filename
    else:
        file_data[date] = {file_type: filename}

scene_specs = pd.DataFrame(file_data).transpose()
#print(scene_specs.head(2))

In [9]:
#######################
##### SCENE SPECS #####
#######################

scene_specs['zone_number'] = ""
scene_specs['upper_left_x'] = ""
scene_specs['upper_left_y'] = ""
scene_specs['lower_right_x'] = ""
scene_specs['lower_right_y'] = ""
scene_specs['satellite'] = ""
scene_specs['year'] = ""
scene_specs['month'] = ""
scene_specs['day'] = ""
scene_specs['season'] = ""
scene_specs['target_date'] = ""

for index, row in scene_specs.iterrows():
    xml_file = row['xml']
    zone_number = extract_utm_zone(os.path.join(folder_path, xml_file))
    corner_coordinates = extract_scene_coords(os.path.join(folder_path, xml_file))
    scene_specs.at[index,'zone_number'] = zone_number
    scene_specs.at[index,'upper_left_x'] = corner_coordinates.get('UL_x')
    scene_specs.at[index,'upper_left_y'] = corner_coordinates.get('UL_y')
    scene_specs.at[index,'lower_right_x'] = corner_coordinates.get('LR_x')
    scene_specs.at[index,'lower_right_y'] = corner_coordinates.get('LR_y')
    scene_specs.at[index,'satellite'] = row['xml'][3:4]
    scene_specs.at[index,'year'] = row['xml'][17:21]
    scene_specs.at[index,'month'] = row['xml'][21:23]
    scene_specs.at[index,'day'] = row['xml'][23:25]
    if scene_specs.at[index, 'month'] in grow_time:
        scene_specs.at[index,'season'] = "gro"
    elif scene_specs.at[index, 'month'] in plant_time:
        scene_specs.at[index,'season'] = "sow"
    else:
        continue

scene_specs['date'] = pd.to_datetime(scene_specs['year'] + scene_specs['month'] + scene_specs['day'], format='%Y%m%d')

for index, row in scene_specs.iterrows():
    if scene_specs.at[index, 'season'] == "gro":
        scene_specs.at[index,'target_date'] = pd.to_datetime(row['year'] + f'-{growing_mid_month}-{growing_mid_day}')
    elif scene_specs.at[index, 'season'] == "sow":
        scene_specs.at[index,'target_date'] = pd.to_datetime(row['year'] + f'-{planting_mid_month}-{planting_mid_day}')

scene_specs['target_date'] = pd.to_datetime(scene_specs['target_date'])
scene_specs['days_from_target'] = (scene_specs['target_date'] - scene_specs['date']).dt.days.abs()
scene_specs.drop(columns=['target_date'], inplace=True)

print(scene_specs.head(4))
print(scene_specs.shape)

                                                   xml  \
20130606  LC08_L2SP_170064_20130606_20200912_02_T1.xml   
20130622  LC08_L2SP_170064_20130622_20200912_02_T1.xml   
20140201  LC08_L2SP_170064_20140201_20200912_02_T1.xml   
20140305  LC08_L2SP_170064_20140305_20200911_02_T1.xml   

                                                    quality  \
20130606  LC08_L2SP_170064_20130606_20200912_02_T1_QA_PI...   
20130622  LC08_L2SP_170064_20130622_20200912_02_T1_QA_PI...   
20140201  LC08_L2SP_170064_20140201_20200912_02_T1_QA_PI...   
20140305  LC08_L2SP_170064_20140305_20200911_02_T1_QA_PI...   

                                                        evi zone_number  \
20130606  LC08_L2SP_170064_20130606_20200912_02_T1_SR_EV...          36   
20130622  LC08_L2SP_170064_20130622_20200912_02_T1_SR_EV...          36   
20140201  LC08_L2SP_170064_20140201_20200912_02_T1_SR_EV...          36   
20140305  LC08_L2SP_170064_20140305_20200911_02_T1_SR_EV...          36   

         upper_le

In [10]:
###############################
## FIND AVERAGE BOUNDING BOX ##
###############################

seven_scenes = scene_specs[(scene_specs['satellite'] == '7') & (scene_specs['season'] == 'gro')]

med_east = seven_scenes['upper_left_x'].median()

seven_scenes['diff_from_median'] = abs(seven_scenes['upper_left_x'] - med_east)
sorted_seven = seven_scenes.sort_values(by='diff_from_median')
top_5_rows = sorted_seven.head(5)#.copy()

med_north = top_5_rows['upper_left_y'].median()
med_scenes = top_5_rows[top_5_rows['upper_left_y'] == med_north]

row_index = med_scenes.index[0]
med_scene = top_5_rows.loc[row_index]#.copy()
easting1 = (med_scene['upper_left_x'] + med_scene['lower_right_x'])/2 - 6000
easting2 = (med_scene['upper_left_x'] + med_scene['lower_right_x'])/2 + 6000
northing1 = (med_scene['upper_left_y'] + med_scene['lower_right_y'])/2 - 6000
northing2 = (med_scene['upper_left_y'] + med_scene['lower_right_y'])/2 + 6000

C:\Users\asobc\AppData\Local\Temp\ipykernel_22764\3624900652.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seven_scenes['diff_from_median'] = abs(seven_scenes['upper_left_x'] - med_east)


In [15]:
#############################
### LOAD AND CLIP RASTERS ###
#############################

# the note indicates which bounding box the trimmed .tif is from.
# ss is the furthest south, ms is middle south, mm is middle, mn is middle north, and nn is furthest north and all are along the central landsat axis
# the slope of the central axis in my test scene was approximately 4.63, so this is the ratio I use

output_folder = f"trimmed_{path}_{row_name}"
output_joined = os.path.join(wd, output_folder)
print(output_joined)
os.makedirs(output_joined, exist_ok=True)

trim_boxes = {
    'ss': (easting1 - 12000, northing1 - 55560, easting2 - 12000, northing2 - 55560),
    'ms': (easting1 - 6000, northing1 - 27780, easting2 - 6000, northing2 - 27780),
    'mm': (easting1, northing1, easting2, northing2),
    'mn': (easting1 + 6000, northing1 + 27780, easting2 + 6000, northing2 + 27780),
    'nn': (easting1 + 12000, northing1 + 55560, easting2 + 12000, northing2 + 55560)
}

loc_list = ['ss','ms','mm','mn','nn']
for i in loc_list:
    trim_box = trim_boxes[i]
    for index, row in scene_specs.iterrows():
        raster_name = row['evi']
        raster_file = os.path.join(folder_path, raster_name)
        season = row['season']
        trim_raster(raster_file, trim_box, output_joined, i, season)

    for index, row in scene_specs.iterrows():
        raster_name = row['quality']
        raster_file = os.path.join(folder_path, raster_name)
        season = row['season']
        trim_raster(raster_file, trim_box, output_joined, i, season)

D:/trimmed_170_064
D:/170064_raw/LC08_L2SP_170064_20130606_20200912_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20130622_20200912_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20140201_20200912_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20140305_20200911_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20140609_20200911_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20140625_20200911_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20150204_20200909_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20150220_20200909_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20150612_20200909_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20160223_20200907_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20160614_20200906_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20160630_20200906_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20170209_20200905_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20170617_20200903_02_T1_SR_EVI.tif
D:/170064_raw/LC08_L2SP_170064_20180212_20200902_02_T1_SR

In [53]:
############################
###### RASTER TO CSV #######
############################

# commented out pieces are for the lon/lat conversion which I will be putting off for this piece

def raster_to_csv(raster_file, quality_file,
                  #east_name, north_name, zone_number,
                  output_joined, id, sat, location, season):

    with rasterio.open(raster_file) as src:
        raster_array = src.read(1)
        nodata_value = src.nodata
        raster_array = np.where(raster_array == nodata_value, np.nan, raster_array)
        transform = src.transform

        rows, cols = raster_array.shape
        X, Y = np.meshgrid(np.arange(0, cols), np.arange(0, rows))
        east, north = transform * (X, Y)

        flat_east = east.flatten()
        flat_north = north.flatten()
        flat_data = raster_array.flatten()

        evi = pd.DataFrame({
            'easting': flat_east,
            'northing': flat_north,
            'evi': flat_data
        })
    with rasterio.open(quality_file) as src:
        quality_array = src.read(1)

        nodata_value = src.nodata
        quality_array = np.where(quality_array == nodata_value, np.nan, quality_array)
        transform = src.transform
        rows, cols = quality_array.shape
        X_q, Y_q = np.meshgrid(np.arange(0, cols), np.arange(0, rows))

        east_q, north_q = transform * (X_q, Y_q)

        flat_east_q = east_q.flatten()
        flat_north_q = north_q.flatten()
        flat_data_q = quality_array.flatten()

        qual = pd.DataFrame({
            'easting': flat_east_q,
            'northing': flat_north_q,
            'quality': flat_data_q
        })

    df = pd.merge(evi, qual, on=['easting', 'northing'])

    del evi
    del qual

    #df[['latitude', 'longitude']] = df.apply(utm_to_latlon, axis=1, args=(east_name, north_name, zone_number))

    # I don't actually need to generate csvs to evaluate, so will probably cut this in the future

    output_file = os.path.join(output_joined,os.path.basename(raster_file)[:-23] + f'{location}.csv')
    #df.to_csv(output_file, index=False)

    size = df.shape[0]
    qual_counts = df['quality'].value_counts()

    index = qual_counts.index.to_numpy()
    values = qual_counts.to_numpy()
    qual_sum = pd.DataFrame({'quality': index, 'count': values})
    qual_sum['date'] = id
    qual_sum['satellite'] = sat
    qual_sum['length'] = size
    qual_sum['season'] = season
    qual_sum['trim_file_evi'] = os.path.basename(raster_file)
    qual_sum['trim_file_qual'] = os.path.basename(quality_file)
    return qual_sum

In [54]:
#################################
### QUALITY LIST TO DATAFRAME ###
#################################

output_folder = f"trimmed_{path}_{row_name}"
output_joined = os.path.join(wd, output_folder)

list_mm = []
list_nn = []
list_ms = []
list_mn = []
list_ss = []

loc_list = ['nn','mm','ms','mn','ss']

for i in loc_list:
    #qual_list = []
    for index, row in scene_specs[scene_specs['season'] == 'gro'].iterrows():
        raster_name = row['evi']
        quality_name = row['quality']
        season = row['season']
        raster_file = os.path.join(output_joined, raster_name[:-4] + f'_{season}_{i}_TRIM.tif')
        quality_file = os.path.join(output_joined, quality_name[:-4] + f'_{season}_{i}_TRIM.tif')
        #zone_number = row['zone_number']
        #east_name = 'easting'
        #north_name = 'northing'
        id = index
        sat = row['satellite']
        location = i
        gro_df = raster_to_csv(raster_file, quality_file,
                                #east_name, north_name, zone_number,
                                output_joined, id, sat, location, season)
        if i == 'mm':
            list_mm.append(gro_df)
        elif i == 'nn':
            list_nn.append(gro_df)
        elif i == 'ms':
            list_ms.append(gro_df)
        elif i == 'mn':
            list_mn.append(gro_df)
        elif i == 'ss':
            list_ss.append(gro_df)

for i in loc_list:
    #qual_list = []
    for index, row in scene_specs[scene_specs['season'] == 'sow'].iterrows():
        raster_name = row['evi']
        quality_name = row['quality']
        season = row['season']
        raster_file = os.path.join(output_joined, raster_name[:-4] + f'_{season}_{i}_TRIM.tif')
        quality_file = os.path.join(output_joined, quality_name[:-4] + f'_{season}_{i}_TRIM.tif')
        #zone_number = row['zone_number']
        #east_name = 'easting'
        #north_name = 'northing'
        id = index
        sat = row['satellite']
        location = i

        sow_df = raster_to_csv(raster_file, quality_file,
                                #east_name, north_name, zone_number,
                                output_joined, id, sat, location, season)
        if i == 'mm':
            list_mm.append(sow_df)
        elif i == 'nn':
            list_nn.append(sow_df)
        elif i == 'ms':
            list_ms.append(sow_df)
        elif i == 'mn':
            list_mn.append(sow_df)
        elif i == 'ss':
            list_ss.append(sow_df)

In [55]:
print(list_mm)

[   quality   count      date satellite  length season  \
0  21824.0  160000  20130606         8  160000    gro   

                                       trim_file_evi  \
0  LC08_L2SP_170064_20130606_20200912_02_T1_SR_EV...   

                                      trim_file_qual  
0  LC08_L2SP_170064_20130606_20200912_02_T1_QA_PI...  ,    quality   count      date satellite  length season  \
0  21824.0  160000  20130622         8  160000    gro   

                                       trim_file_evi  \
0  LC08_L2SP_170064_20130622_20200912_02_T1_SR_EV...   

                                      trim_file_qual  
0  LC08_L2SP_170064_20130622_20200912_02_T1_QA_PI...  ,    quality   count      date satellite  length season  \
0  21824.0  158968  20140609         8  160000    gro   
1  54596.0    1032  20140609         8  160000    gro   

                                       trim_file_evi  \
0  LC08_L2SP_170064_20140609_20200911_02_T1_SR_EV...   
1  LC08_L2SP_170064_20140609_20200911

In [57]:
############################
## EVALUATE SCENE QUALITY ##
############################

import pandas as pd

starting_year = 1984
ending_year = 2023
years_range = np.arange(starting_year, ending_year + 1).tolist()

full_list = {'ss': list_ss, 'ms': list_ms, 'mm': list_mm, 'mn': list_mn, 'nn': list_nn}
#list_sow = {'ss': sow_list_ss, 'ms': sow_list_ms, 'mm': sow_list_mm, 'mn': sow_list_mn, 'nn': sow_list_nn}

max_unique_length = 0
best_list_name = None
for name, list in full_list.items():
    # making the basic dataframe out of each list
    result_df = pd.concat(list, axis=0)
    result_df['year'] = result_df['date'].str[0:4]#.astype(int)
    result_df['month'] = result_df['date'].str[4:6]
    result_df['day'] = result_df['date'].str[6:8]
    result_df['date'] = pd.to_datetime(result_df['date'], format='%Y%m%d')
    result_df.reset_index(drop=True, inplace=True)
    for index, row in result_df.iterrows():
        year = row['year']
        if row['season'] == 'gro':
            result_df.at[index,'target_date'] = pd.to_datetime(f'{year}{growing_mid_month}{growing_mid_day}',format='%Y%m%d')
            #print(result_df.at[index,'target_date'])
        elif row['season'] == 'sow':
            result_df.at[index,'target_date'] = pd.to_datetime(f'{year}{planting_mid_month}{planting_mid_day}', format='%Y%m%d')
        else:
            continue
    #result_df['target_date'] = pd.to_datetime(result_df['target_date'])
    result_df['date'] = pd.to_datetime(result_df['date'])
    result_df['days_from_target'] = (result_df['target_date'] - result_df['date']).dt.days.abs()
    #result_df['days_from_target'] = result_df['days_from_target'].abs()
    #result_df.drop(columns=['target_date'], inplace=True)
    result_df['indicator'] = ""
    # evaluating quality by first categorizing as good or bad
    good = [5440, 21824, 5442, 21826, 5504, 21888, 5506, 21890, 5760, 22144, 7824, 24472, 24216, 8088]
    result_df['indicator'] = np.where(result_df['quality'].isin(good), 1, 0)
    result_df['qual_x_ind'] = result_df['indicator'] * result_df['count']
    result_df['qual_x_ind_sum'] = result_df.groupby(['date', 'satellite'])['qual_x_ind'].transform('sum')
    result_df['year_good_gro'] = np.where((result_df['qual_x_ind_sum'] >= 128000) & (result_df['season'] == 'gro'), result_df['year'], np.nan)
    result_df['year_good_sow'] = np.where((result_df['qual_x_ind_sum'] >= 128000) & (result_df['season'] == 'sow'), result_df['year'], np.nan)
    unique_gro = result_df['year_good_gro'].unique().tolist()
    unique_sow = result_df['year_good_sow'].unique().tolist()
    series_gro = pd.Series(unique_gro)
    series_gro = series_gro.dropna()
    series_gro = series_gro.astype(float)
    gro_list = series_gro.tolist()
    gro_list = [int(element) for element in gro_list]
    series_sow = pd.Series(unique_sow)
    series_sow = series_sow.dropna()
    series_sow = series_sow.astype(float)
    sow_list = series_sow.tolist()
    sow_list = [int(element) for element in sow_list]
    int_list= [value for value in sow_list if value in gro_list]
    int_set = set(gro_list) & set(sow_list)
    # if len(unique_list) > max_unique_length:
    #     max_unique_length = len(unique_list)
    #     best_list_name = name
    missing_elements = [year for year in years_range if year not in int_list]
    result_df['year_in_both_seasons'] = np.where(result_df['year'].astype(int).isin(int_set), 1, np.nan)
    print(f'For {name}, from {starting_year} to {ending_year}, there are {len(int_list)} years available with low cloud cover.')
    #print(f'Included years: {sorted(unique_list)}')
    print(f'Missing years for {name}: {sorted(missing_elements)}')
    # writing the csvs
    qual_folder = f'{wd}{path}_{row_name}_qual'
    os.makedirs(qual_folder, exist_ok=True)
    qual_out = os.path.join(wd, qual_folder, f'{path}_{row_name}_quality_{name}_full.csv')
    result_df.to_csv(qual_out, index=False)

For ss, from 1984 to 2023, there are 22 years available with low cloud cover.
Missing years for ss: [1984, 1985, 1986, 1987, 1988, 1989, 1991, 1992, 1994, 1996, 1997, 1998, 2000, 2001, 2003, 2006, 2011, 2017]
For ms, from 1984 to 2023, there are 22 years available with low cloud cover.
Missing years for ms: [1984, 1985, 1986, 1987, 1988, 1989, 1991, 1992, 1994, 1996, 1997, 1998, 2000, 2001, 2005, 2006, 2008, 2011]
For mm, from 1984 to 2023, there are 21 years available with low cloud cover.
Missing years for mm: [1984, 1985, 1986, 1987, 1988, 1989, 1991, 1992, 1994, 1996, 1997, 1998, 2000, 2001, 2003, 2006, 2011, 2019, 2022]
For mn, from 1984 to 2023, there are 20 years available with low cloud cover.
Missing years for mn: [1984, 1985, 1986, 1987, 1988, 1989, 1991, 1992, 1994, 1995, 1996, 1997, 1998, 2000, 2001, 2003, 2005, 2006, 2011, 2022]
For nn, from 1984 to 2023, there are 24 years available with low cloud cover.
Missing years for nn: [1984, 1985, 1986, 1987, 1988, 1989, 1991, 199

In [60]:
###########################
##### PICK BEST SCENE #####
###########################
best_bb = 'nn'
best_path = os.path.join(qual_folder, f'{path}_{row_name}_quality_{best_bb}_full.csv')
best_df = pd.read_csv(best_path)
best_df = best_df.dropna(subset=['year_good_sow', 'year_good_gro'], how='all')
best_df = best_df.dropna(subset=['year_in_both_seasons'])
best_df['min_days_from_target'] = best_df.groupby(['year', 'season'])['days_from_target'].transform('min')
best_df = best_df[best_df['days_from_target'] <= best_df['min_days_from_target']]
unique_evi = best_df['trim_file_evi'].unique().tolist()
unique_qual = best_df['trim_file_qual'].unique().tolist()
unique_scenes = unique_evi + unique_qual
best_df = best_df.drop(columns=['min_days_from_target', 'year_in_both_seasons'])
#print(best_df.head(10))
#print(unique_scenes)
best_df.to_csv(os.path.join(qual_folder,f'{path}_{row_name}_{best_bb}_selected.csv'), index = False)

def save_list_to_txt(lst, filename):
    with open(filename, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')

# Save the list to a .txt file
save_list_to_txt(unique_qual, os.path.join(qual_folder,f'{path}_{row_name}_{best_bb}_quality_scenes.txt'))
save_list_to_txt(unique_evi, os.path.join(qual_folder,f'{path}_{row_name}_{best_bb}_evi_scenes.txt'))
save_list_to_txt(unique_scenes, os.path.join(qual_folder,f'{path}_{row_name}_{best_bb}_all_scenes.txt'))